In [1]:
# Для работы с большими датасетами используем библиотеку cudf от NVIDIA, с целью перенаправления вычислений на GPU

import cudf as pd
# import cupy as cp
import numpy as np
# import matplotlib.pyplot as plt
# import pandas

In [2]:
# Загружаем исходные данные

adv_data = pd.read_csv('data/adv_data.csv')
users_data = pd.read_csv('data/users_data.csv')

In [3]:
# adv_data['campaign_type'].unique()

In [4]:
# Переменная, в которой хранится значение курса доллара США на текущий день, для приведения колонок campaign_cost и payout к общему рублёвому значению

usdrub = 90.41

In [5]:
# Метод, приводящий значения в долларах США к рублевым значениям в столбце payout

def convert_to_rub(row):
    x = row['payout']
    if row['currency'] == 'USD':       
        return x * usdrub
    else:
        return x

In [6]:
# Метод, приводящий значения в долларах США к рублевым значениям в столбце campaign_cost

def convert_campaign_cost_to_rub(row):
    x = row['campaign_cost']
    if row['currency'] == 'USD':       
        return x * usdrub
    else:
        return x

In [7]:
# Применение вышеуказанных методов

adv_data['campaign_cost'] = adv_data.apply(convert_campaign_cost_to_rub, axis=1)

In [8]:
# Удаление столбцов currency из датасетов event_data, adv_data

adv_data.drop('currency', inplace=True, axis=1)

In [9]:
# Экспериментально установили, что неуникальные значения пользовательских данных - это значения, в которых в ячейке 'user_id' записано значение NaN

users_data = users_data[~users_data['user_id'].isna()]

In [10]:
# Заполнили значения NaN в столбце campaign_cost нулями

adv_data['campaign_cost'].fillna(0.0, inplace=True)

In [11]:
# Экспериментально установили, что столбец delete_dtm полностью пустой, поэтому удаляю его

users_data.drop('delete_dtm', axis=1, inplace=True)

In [12]:
# Из-за того, что номера телефонов в таблицах event_data, users_data записаны по-разному (со знаком + и без него) в следующих трех ячейках приводим телефоны к единому написанию:

# 1) создаем numpy-массивы с нулевыми значениями, по размеру идентичные размеру столбцов user_phone в вышеуказанных датасетах

user_phones = users_data['user_phone'].to_arrow()
user_test = np.zeros_like(user_phones)

In [13]:
# 2) заполняем массивы строковыми значениями номеров телефонов без знаков +

for i, val in enumerate(user_phones):
    user_test[i] = str(val).strip('+')

In [14]:
# 3) заменяем исходные значения в датасетах заполненными массивами


users_data['user_phone'] = user_test

In [15]:
# Для экономии памяти удаляем ненужные массивы

del user_test

In [16]:
# Создаем массив уникальных значений ID пользователей

users_uuids = users_data['user_id'].unique().to_arrow()

In [17]:
# Из датасета adv_data выбираем значения, которым соответствуют значения пользователей

adv_data = adv_data[adv_data['user_id'].isin(users_uuids)]

In [18]:
# Проверяем оставшиеся после этого значения в таблице adv_data

# adv_data.info()

In [19]:
# Создаем массив уникальных значений телефонов пользователей

user_phones = users_data['user_phone'].unique().to_arrow()

In [20]:
# users_data.info()

In [21]:
# event_data.info()

In [22]:
# проверяем оставшиеся после этого значения в таблице adv_data

# event_data.info()

In [23]:
# Соединяем таблицы adv_data, users_data по ID пользователей аналогично SQL-методу left-join (остаются все значения из датасета adv_data и соответствующие им значения из 
# датасета users_data)

adv_data_plus_user_data = adv_data.merge(users_data, how='left', on='user_id')

In [24]:
# Проверяем новый датасет

# adv_data_plus_user_data.info()

In [25]:
# Для экономии памяти удаляем ненужные массивы и таблицы

del users_uuids, adv_data, users_data

In [26]:
# adv_data_plus_user_data.loc[adv_data_plus_user_data['user_phone'] == '79101867847']

In [27]:
# event_data.loc[event_data['user_phone'] == '79101867847']

In [28]:
adv_data_plus_user_data_with_dummies = pd.get_dummies(adv_data_plus_user_data, columns=['campaign_type', 'source_medium', 'interface'], dtype='int')[['user_phone', 'campaign_cost', 
       'campaign_type_click', 'campaign_type_conversion', 'campaign_type_show',
       'source_medium_android / mobile_app', 'source_medium_bing / organic',
       'source_medium_email / email', 'source_medium_finance.rambler.ru / cpm',
       'source_medium_google / organic', 'source_medium_ios / mobile_app',
       'source_medium_not-set / none', 'source_medium_pikabu.ru / cpm',
       'source_medium_regular / email', 'source_medium_regular / sms',
       'source_medium_ru.android.bankproduct / mobile_app',
       'source_medium_ru.android.bankproduct_rustore / mobile_app',
       'source_medium_trigger / sms', 'source_medium_vk / cpc',
       'source_medium_web.telegram.org / referral ',
       'source_medium_ya.ru / cpc', 'source_medium_yahoo / organic',
       'source_medium_yandex / cpc', 'source_medium_yandex / organic',
       'source_medium_zen.yandex.ru / cpm', 'interface_api', 'interface_app',
       'interface_web']]

In [29]:
adv_data_plus_user_data_with_dummies.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 9999908 entries, 0 to 9999907
Data columns (total 28 columns):
 #   Column                                                     Dtype
---  ------                                                     -----
 0   user_phone                                                 object
 1   campaign_cost                                              float64
 2   campaign_type_click                                        int64
 3   campaign_type_conversion                                   int64
 4   campaign_type_show                                         int64
 5   source_medium_android / mobile_app                         int64
 6   source_medium_bing / organic                               int64
 7   source_medium_email / email                                int64
 8   source_medium_finance.rambler.ru / cpm                     int64
 9   source_medium_google / organic                             int64
 10  source_medium_ios / mobile_app       

In [30]:
del adv_data_plus_user_data

In [31]:
adv_data_plus_user_data_with_dummies = adv_data_plus_user_data_with_dummies.sort_values(by='user_phone')

In [32]:
adv_data_plus_user_data_with_dummies_1 = adv_data_plus_user_data_with_dummies.iloc[0:2500000].groupby('user_phone').sum()

In [33]:
adv_data_plus_user_data_with_dummies_2 = adv_data_plus_user_data_with_dummies.iloc[2500000:5000000].groupby('user_phone').sum()

In [34]:
adv_data_plus_user_data_with_dummies_3 = adv_data_plus_user_data_with_dummies.iloc[5000000:7500000].groupby('user_phone').sum()

In [35]:
adv_data_plus_user_data_with_dummies_4 = adv_data_plus_user_data_with_dummies.iloc[7500000:].groupby('user_phone').sum()

In [36]:
adv_data_plus_user_data_with_dummies_all = pd.concat([
    adv_data_plus_user_data_with_dummies_1,
    adv_data_plus_user_data_with_dummies_2
])

In [37]:
del adv_data_plus_user_data_with_dummies_1, adv_data_plus_user_data_with_dummies_2

In [38]:
adv_data_plus_user_data_with_dummies_all_1 = pd.concat([
    adv_data_plus_user_data_with_dummies_3,
    adv_data_plus_user_data_with_dummies_4
])

In [39]:
del adv_data_plus_user_data_with_dummies_3, adv_data_plus_user_data_with_dummies_4

In [40]:
adv_data_plus_user_data_with_dummies_all_final = pd.concat([
    adv_data_plus_user_data_with_dummies_all,
    adv_data_plus_user_data_with_dummies_all_1
])

In [41]:
del adv_data_plus_user_data_with_dummies_all, adv_data_plus_user_data_with_dummies_all_1

In [43]:
adv_data_plus_user_data_with_dummies_all_final.to_csv('data/adv_data_plus_users_groupped.csv')

In [44]:
del adv_data_plus_user_data_with_dummies_all_final

In [45]:
event_data = pd.read_csv('data/event_data.csv')

In [46]:
event_data['payout'] = event_data.apply(convert_to_rub, axis=1)
event_data.drop('currency', inplace=True, axis=1)
event_data['payout'].fillna(0.0, inplace=True)

event_phones = event_data['user_phone'].to_arrow()
np_test = np.zeros_like(event_phones)

for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')
    
event_data['user_phone'] = np_test

del np_test

event_data = event_data[event_data['user_phone'].isin(user_phones)]

In [47]:
event_data.drop(['id', 'event_dtm', 'update_dtm'], inplace=True, axis=1)

In [48]:
event_data.head(1)

,user_phone,event_type,event_status,payout,product_type
0,79140092454,show,<NA>,0.0,mortgage


In [49]:
event_data_with_dummies = pd.get_dummies(event_data, columns=['event_type', 'product_type', 'event_status'])

In [50]:
del event_data, user_phones

In [51]:
event_data_with_dummies_groupped = event_data_with_dummies.groupby('user_phone').sum()

In [52]:
del event_data_with_dummies

In [53]:
event_data_with_dummies_groupped.to_csv('data/event_data_with_dummies_groupped.csv')

In [54]:
del event_data_with_dummies_groupped

In [55]:
event_data_with_dummies_groupped = pd.read_csv('data/event_data_with_dummies_groupped.csv')

In [56]:
adv_data_plus_user_data_with_dummies_groupped = pd.read_csv('data/adv_data_plus_users_groupped.csv')

In [57]:
all_data = adv_data_plus_user_data_with_dummies_groupped.merge(event_data_with_dummies_groupped, how='inner', on='user_phone')

In [58]:
del event_data_with_dummies_groupped, adv_data_plus_user_data_with_dummies_groupped

In [59]:
all_data.to_csv('data/all_data_groupped.csv')

In [60]:
del all_data